# Pattern Discovery MP1

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
from collections import Counter

## Read

In [2]:
categories = pd.read_table('categories.txt', header=None, names=['category'])
categories.shape

(77185, 1)


In [11]:
sup = 0.01
categories_list = categories.category.str.split(";")
count = Counter(category for category_list in categories_list for category in category_list)
n = categories.shape[0]
item_n = np.sum(list(zip(*count.items()))[1])
min_sup = np.floor(n*sup)

# dynamic variable
vars()['join5'] = 2

# scan
scan1 = dict()
[scan1.update({key: value}) for key, value in count.items() if value > min_sup]
print("scan1 done")

# generate self-join
join1 = [comb for comb in combinations(scan1.keys(), 2)]
count = Counter(comb for category_list in categories_list for comb in combinations(category_list,2) for join in join1 if comb==join)

# scan phase 2
scan2= dict()
[scan2.update({key: value}) for key, value in count.items() if value > min_sup]
print("scan2 done")

done
done


In [127]:
# to list of sets
categories_list = [category_list for category_list in categories.category.str.split(';')]
count = Counter(category for category_list in categories_list for category in category_list)

# scan
scan1 = dict()
[scan1.update({key: value}) for key, value in count.items() if value > min_sup]

# self-join list
join = [comb for comb in combinations(scan1.keys(),2)]

In [257]:
%%time
data = categories

# set minimum support
n = data.shape[0]
min_sup = np.floor(n*sup)

# to list of sets
categories_list = [set(category_list) for category_list in data.category.str.split(';')]

# result container
rules = dict()

# count items
count = Counter(category for category_list in categories_list for category in category_list)
scan = dict()
[scan.update({key: value}) for key, value in count.items() if value > min_sup]

# join
join = Counter(comb for category_list in categories_list for comb in combinations(category_list, 2))

Wall time: 589 ms


In [158]:
%%time
data = categories

# set minimum support
n = data.shape[0]
min_sup = np.floor(n*sup)

# to list of sets
categories_list = [set(category_list) for category_list in data.category.str.split(';')]

# result container
rules = dict()

# count items
count = Counter(category for category_list in categories_list for category in category_list)
scan = dict()
[scan.update({key: value}) for key, value in count.items() if value > min_sup]

i = 1
while(True):
    # get join of categories_list
    join = Counter(comb for category_list in categories_list for comb in combinations(category_list, i))
    [rules.update({key: value}) for key, value in join.items() if value > min_sup]
    i += 1
    if not len(join)>0:
        break
rules

Wall time: 103 ms


## Apriori

In [162]:
def apriori(data, sup = 0.01):
    # set minimum support
    n = data.shape[0]
    min_sup = np.floor(n*sup)

    # to list of sets
    categories_list = [set(category_list) for category_list in data.category.str.split(';')]
    
    # result container
    rules = dict()
    
    i = 1
    while(True):
        # get join of categories_list
        join = Counter(comb for category_list in categories_list for comb in combinations(category_list, i))
        [rules.update({key: value}) for key, value in join.items() if value > min_sup]
        i += 1
        if not len(join)>0:
            break
    return rules

In [163]:
rules = apriori(categories)

In [194]:
# Part 1
keys = list(zip(*rules.items()))[0]
keys = [key for key in keys if len(key)==1]

one_rules = dict()
for key in keys:
    one_rules.update({key: rules.get(key)})

In [224]:
# save
with open('patterns1.txt', 'w') as file:
    for key, value in one_rules.items():
        file.write(str(value) + ":" + str(key[0])+'\n')

In [225]:
# save Part 2
with open('patterns2.txt', 'w') as file:
    for key, value in rules.items():
        file.write(str(value)+':'+';'.join(key) +'\n')